<a href="https://www.kaggle.com/code/ub2002/text-to-json-llm?scriptVersionId=251551354" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
key = user_secrets.get_secret("Token")

In [3]:
MODEL_ID="google/gemma-2-9b-it"
input_text = """# "MkDocs Publisher" Action

I keep repeating the steps to build and deploy our MkDocs documentation sites to GitHub Pages across different repos. Let's create a reusable composite action to handle this.

Action Name: MkDocs Publisher
Purpose: A simple action to build an MkDocs site and push it to the gh-pages branch. Should be easy to use. Author should be listed as 'DevRel Team'.

Inputs Needed:

python-version: We need to specify the Python version for setting up the environment. Users should be able to choose. Let's make this optional and default it to 3.11. Description: 'The version of Python to set up for building.'
requirements-file: Users might have different requirements files (e.g., requirements.txt, docs/requirements.txt). This input should specify the path. It's required. Description: 'Path to the Python requirements file'.
gh-token: The GitHub token for pushing to gh-pages. This is absolutely required. Description: 'GitHub token for deployment.' Let's add a note that this is usually ${{ secrets.GITHUB_TOKEN }}. We should probably add a deprecation message to this input later if we find a better way, maybe: "Prefer using GITHUB_TOKEN environment variable directly if permissions allow."
Outputs:

The action needs to output the URL where the site was deployed. Let's call this output page-url. Its description should be 'The URL of the deployed GitHub Pages site.' The actual value should come from the deployment step (see below).

How it Runs (Execution):

This will be a composite action (using: composite). Here are the steps involved:

Checkout Code: First, we need the repository code. Use the standard actions/checkout@v4.
Setup Python: Next, set up the Python environment. Use actions/setup-python@v5. We must use the python-version input provided by the user here. Let's ID this step as setup_python.
Install Dependencies: Run a command to install the Python packages. The command is pip install -r ${{ inputs.requirements-file }}. Execute this using the bash shell. Maybe give this step the name "Install Python Packages".
Build Site: Run the command mkdocs build. Use bash for this too.
Deploy to Pages: Use an existing action for the deployment. peaceiris/actions-gh-pages@v3 seems popular. Give this step the ID deploy. We need to configure it:
Set its github_token parameter to the gh-token input we defined earlier.
Set its publish_dir parameter to ./site (which is the default build output dir for MkDocs).
Maybe add a condition (if:): github.ref == 'refs/heads/main' so it only deploys from the main branch.

Remember the page-url output? Its value needs to be linked to the output of the deploy step. So, for the page-url output, the value should be ${{ steps.deploy.outputs.page_url }}.

Branding:
For the marketplace look, let's use the color blue and the book-open icon. Looks professional.
"""
with open("/kaggle/input/schema/schema.json", "r") as f:
  json_schema = json.load(f)


json_schema_str = json.dumps(json_schema, indent=2)


prompt = f'''Below is a text paired with input that provides further context. Write JSON output only json code as output.
### Input:
{input_text}
### Schema:
{json_schema_str}
### Response:
'''

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=key)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=key,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [5]:

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output_tokens = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        temperature=0.2,
        eos_token_id=tokenizer.eos_token_id,
)

response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
W0720 18:50:46.192000 19 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
W0720 18:50:46.204000 19 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)


In [6]:
response_after_marker = response.split("### Response:")[-1].strip()
first_brace = response_after_marker.find('{')
last_brace = response_after_marker.rfind('}')
json_str = response_after_marker[first_brace:last_brace + 1]
json_obj = json.loads(json_str)

print(json.dumps(json_obj, indent=2))

{
  "action_name": "MkDocs Publisher",
  "purpose": "A simple action to build an MkDocs site and push it to the gh-pages branch. Should be easy to use.",
  "author": "DevRel Team",
  "inputs": {
    "python-version": {
      "required": false,
      "default": "3.11",
      "description": "The version of Python to set up for building."
    },
    "requirements-file": {
      "required": true,
      "default": null,
      "description": "Path to the Python requirements file"
    },
    "gh-token": {
      "required": true,
      "default": null,
      "description": "GitHub token for deployment. Usually ${{ secrets.GITHUB_TOKEN }}. Prefer using GITHUB_TOKEN environment variable directly if permissions allow."
    }
  },
  "outputs": {
    "page-url": {
      "description": "The URL of the deployed GitHub Pages site.",
      "value_from": "${{ steps.deploy.outputs.page_url }}"
    }
  },
  "runs": {
    "using": "composite",
    "steps": [
      {
        "name": "Checkout Code",
       

In [7]:
!pip install -q gradio

In [8]:

import gradio as gr


# Assuming schema.json exists and contains the schema you want to use
try:
    with open("/kaggle/input/schema/schema.json", "r") as f:
        json_schema = json.load(f)
except FileNotFoundError:
    # Create a dummy schema.json if it doesn't exist for demonstration
    print("Schema file not found. Creating a dummy schema.")


json_schema_str = json.dumps(json_schema, indent=2)


def generate_json_from_text(input_text):
    prompt = f'''Below is a text paired with input that provides further context. Write JSON output only json outpu.
### Input:
{input_text}
### Schema:
{json_schema_str}
### Response:
'''
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=False,
            temperature=0.2,
            eos_token_id=tokenizer.eos_token_id,
        )
    response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    # Extract only the JSON part from the response if the model includes extra text
    try:
        response_after_marker = response.split("### Response:")[-1].strip()
        first_brace = response_after_marker.find('{')
        last_brace = response_after_marker.rfind('}')
        json_str = response_after_marker[first_brace:last_brace + 1]
        json_obj = json.loads(json_str)
        return json.dumps(json_obj, indent=2)
    except json.JSONDecodeError:
        return "Could not parse JSON from model output:\n" + response


iface = gr.Interface(
    fn=generate_json_from_text,
    inputs=gr.Textbox(lines=20, placeholder="Enter your text here...", label="Input Text"),
    outputs=gr.Textbox(lines=20, label="Generated JSON Output"),
    title="Text to JSON with Gemma",
    description="Enter text and the model will attempt to generate JSON based on a provided schema."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://362f7ed29fe5cd0bb1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
